In [ ]:
import os
import numpy as np
from scipy import fftpack
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import cm
import pypsa

from PIL import Image

mpl.use('Agg')

In [ ]:
def colorFader(c1,c2,mix=0): #fade (linear interpolate) from color c1 (at mix=0) to c2 (mix=1)
    c1=np.array(mpl.colors.to_rgb(c1))
    c2=np.array(mpl.colors.to_rgb(c2))
    return mpl.colors.to_hex((1-mix)*c1 + mix*c2)

In [ ]:
#fig,ax=plt.subplots(figsize=(5,5))

x = np.linspace(-10., 10., 2000)
y = np.sin(x) * np.exp(-(x/5.0)**2)

c1 = '#14a951'
c2 = '#d9a717'
c3 = '#258fb0'

plt.scatter(x,y, s=1, color=[colorFader(c2,c3,mix=h) for h in np.arange(0,1,1/2000)], zorder=8)
plt.scatter([-10, 10], [1e-2,-1e-2], s=50, color=[c2, c3])

In [ ]:
framewidth=500
stepsize=50

color_left = c2

imname = 0
for c2 in ['#258fb0']:
    for i in range(0,1):
        fig,ax=plt.subplots()
        ax.axis('off')
    
        plt.plot(x,y,linewidth=.5, color='#c81952', zorder=0)    
        plt.scatter([-10, 10], [1e-2,-1e-2], s=50, color=[color_left, c2])
    
        plt.savefig(f'../header-{imname}.png', bbox_inches='tight')
        imname += 1
             
    for i in range(-framewidth,2000+framewidth,stepsize):
        
        start = i
        end = i+framewidth
        if start<0: start=0
        if end>2000: end=2000
        
        framesize = len(x[start:end])
        
        fig,ax=plt.subplots()
        ax.axis('off')
    
        plt.plot(x,y,linewidth=.5, color='#c81952', zorder=0)
    
        color=[colorFader(color_left, c2, mix=h) for h in np.arange(start, end)/(2000+2*framewidth)]
    
        ax.scatter(x[start:end], y[start:end], color=color, zorder=8)
        plt.scatter([-10, 10], [1e-2,-1e-2], s=50, color=[color_left, c2])
    
        plt.savefig(f'../header-{imname}.png', bbox_inches='tight')
        imname += 1
    
#plt.show()

In [ ]:
def gen_frame(path):
    im = Image.open(path)
    alpha = im.getchannel('A')

    # Convert the image into P mode but only use 255 colors in the palette out of 256
    im = im.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)

    # Set all pixel values below 128 to 255 , and the rest to 0
    mask = Image.eval(alpha, lambda a: 0 if a <=128 else 0)

    # Paste the color of index 255 and use alpha as a mask
    im.paste(0, mask)

    # The transparency index is 255
    im.info['transparency'] = 0
    return im


im1 = gen_frame('../header-0.png')
append_images = [f"../header-{i}.png" for i in range(0,imname)]

im1.save('../gifs/header-gif.gif', save_all=True, append_images=[gen_frame(append_images[i]) for i in range(len(append_images))],
         disposal=2, duration=50)

In [ ]:
# delete images...
for file in [f"../header-{i}.png" for i in range(0,imname)]:
    os.remove(file)